In [2]:
## Importing the required libraries
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

In [8]:
# Create SQLite connection
def create_database_connection():
    conn = sqlite3.connect('career_lens.db')
    return conn

In [10]:
# File paths
SALARY_DATA_PATH = r".\Datasets\Salary Data.csv"
ROO_DATA_PATH = r".\Datasets\roo_data.csv"
ROLES_SKILLS_PATH = r".\Datasets\Job opportunities.xlsx"

In [12]:
def import_salary_data(SALARY_DATA_PATH):
    """Import and clean salary dataset"""
    try:
        df = pd.read_csv(SALARY_DATA_PATH)
        
        # Data cleaning
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
        
        # Validate data types
        df['age'] = pd.to_numeric(df['age'], errors='coerce')
        df['years_of_experience'] = pd.to_numeric(df['years_of_experience'], errors='coerce')
        df['salary'] = pd.to_numeric(df['salary'], errors='coerce')
        
        return df
    except Exception as e:
        print(f"Error importing salary data: {e}")
        return None

In [14]:
salary_data = import_salary_data(SALARY_DATA_PATH)

In [16]:
salary_data.head()

,age,gender,education_level,job_title,years_of_experience,salary
0,32.0,Male,Bachelor's,Software Engineer,5.0,90000.0
1,28.0,Female,Master's,Data Analyst,3.0,65000.0
2,45.0,Male,PhD,Senior Manager,15.0,150000.0
3,36.0,Female,Bachelor's,Sales Associate,7.0,60000.0
4,52.0,Male,Master's,Director,20.0,200000.0


In [18]:
def import_roo_data(ROO_DATA_PATH):
    """Import and clean career prediction dataset"""
    try:
        df = pd.read_csv(ROO_DATA_PATH)
        
        # Clean column names
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
        
        # Convert percentage columns to numeric
        percentage_columns = [col for col in df.columns if 'percentage' in col.lower()]
        for col in percentage_columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        return df
    except Exception as e:
        print(f"Error importing roo data: {e}")
        return None

In [20]:
roo_data = import_roo_data(ROO_DATA_PATH)

In [22]:
roo_data.head()

,acedamic_percentage_in_operating_systems,percentage_in_algorithms,percentage_in_programming_concepts,percentage_in_software_engineering,percentage_in_computer_networks,percentage_in_electronics_subjects,percentage_in_computer_architecture,percentage_in_mathematics,percentage_in_communication_skills,hours_working_per_day,...,interested_type_of_books,salary_range_expected,in_a_realtionship?,gentle_or_tuff_behaviour?,management_or_technical,salary/work,hard/smart_worker,worked_in_teams_ever?,introvert,suggested_job_role
0,69,63,78,87,94,94,87,84,61,9,...,Prayer books,salary,no,stubborn,Management,salary,hard worker,yes,no,Database Developer
1,78,62,73,60,71,70,73,84,91,12,...,Childrens,salary,yes,gentle,Technical,salary,hard worker,no,yes,Portal Administrator
2,71,86,91,87,61,81,72,72,94,11,...,Travel,Work,no,stubborn,Management,work,hard worker,no,yes,Portal Administrator
3,76,87,60,84,89,73,62,88,69,7,...,Romance,Work,yes,gentle,Management,work,smart worker,yes,yes,Systems Security Administrator
4,92,62,90,67,71,89,73,71,73,4,...,Cookbooks,salary,no,stubborn,Management,work,hard worker,yes,yes,Business Systems Analyst


In [24]:
roo_data.columns

Index(['acedamic_percentage_in_operating_systems', 'percentage_in_algorithms',
       'percentage_in_programming_concepts',
       'percentage_in_software_engineering', 'percentage_in_computer_networks',
       'percentage_in_electronics_subjects',
       'percentage_in_computer_architecture', 'percentage_in_mathematics',
       'percentage_in_communication_skills', 'hours_working_per_day',
       'logical_quotient_rating', 'hackathons', 'coding_skills_rating',
       'public_speaking_points', 'can_work_long_time_before_system?',
       'self-learning_capability?', 'extra-courses_did', 'certifications',
       'workshops', 'talenttests_taken?', 'olympiads',
       'reading_and_writing_skills', 'memory_capability_score',
       'interested_subjects', 'interested_career_area', 'job/higher_studies?',
       'type_of_company_want_to_settle_in?',
       'taken_inputs_from_seniors_or_elders', 'interested_in_games',
       'interested_type_of_books', 'salary_range_expected',
       'in_a_real

In [26]:
def import_roles_skills_data(ROLES_SKILLS_PATH):
    """Import and clean IT jobs dataset"""
    try:
        # Read the dataset
        df = pd.read_excel(ROLES_SKILLS_PATH)
        
        # Select columns including experience_level
        df = df[['Job Title', 'Required Skills', 'Experience Level']]
        
        # Clean column names
        df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
        
        def clean_skills(skills_str):
            if pd.isna(skills_str):
                return ""
            
            # Split skills by comma and handle possible variations
            skills_list = [skill.strip() for skill in skills_str.split(',')]
            
            # Remove duplicates while preserving order
            seen = set()
            unique_skills = []
            for skill in skills_list:
                skill_lower = skill.lower()
                if skill_lower not in seen:
                    seen.add(skill_lower)
                    unique_skills.append(skill)
            
            return ', '.join(unique_skills)
        
        # Clean skills
        df['required_skills'] = df['required_skills'].apply(clean_skills)
        
        # Group by job title and experience level
        def combine_group_data(group):
            return pd.Series({
                'required_skills': ', '.join(set(
                    skill.strip()
                    for skills in group['required_skills']
                    for skill in skills.split(',')
                    if skill.strip()
                )),
                'experience_level': ', '.join(set(group['experience_level']))
            })
        
        # Group by job title while preserving experience level
        grouped_df = df.groupby('job_title').apply(combine_group_data).reset_index()
        
        return grouped_df
        
    except Exception as e:
        print(f"Error importing IT jobs data: {e}")
        return None

In [ ]:
roles_skills_data = import_roles_skills_data(ROLES_SKILLS_PATH)

In [29]:
roles_skills_data.head()

,job_title,required_skills,experience_level
0,AI Ethics Consultant,"Ethics, AI Governance","Mid-Level, Senior"
1,AI/ML Engineer,"TensorFlow, Python, AI/ML",Mid-Level
2,Blockchain Developer,"Solidity, Blockchain",Mid-Level
3,Business Analyst,"Requirements, Business Analysis",Mid-Level
4,Cloud Architect,"Azure, Cloud Architecture, AWS, Cloud Design",Senior


## Importing the data into sqllite tables

In [33]:
# Create table for IT jobs and skills
create_jobs_table = '''
CREATE TABLE IF NOT EXISTS job_skills_mapping (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    job_title TEXT NOT NULL,
    experience_level TEXT NOT NULL,
    required_skills TEXT NOT NULL
)
'''

# Create table for salary predictions
create_salary_table = '''
CREATE TABLE IF NOT EXISTS salary_prediction (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    age INTEGER,
    gender TEXT,
    education_level TEXT,
    job_title TEXT,
    years_of_experience INTEGER,
    salary FLOAT
)
'''

# Create table for career predictions
create_career_table = '''
CREATE TABLE IF NOT EXISTS career_prediction (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    academic_os_percentage FLOAT,
    algorithms_percentage FLOAT,
    programming_concepts_percentage FLOAT,
    software_engineering_percentage FLOAT,
    computer_networks_percentage FLOAT,
    electronics_subjects_percentage FLOAT,
    computer_architecture_percentage FLOAT,
    mathematics_percentage FLOAT,
    communication_skills_percentage FLOAT,
    hours_working_per_day INTEGER,
    logical_quotient_rating INTEGER,
    hackathons INTEGER,
    coding_skills_rating INTEGER,
    public_speaking_points INTEGER,
    can_work_long_time TEXT,
    self_learning_capability TEXT,
    extra_courses_done INTEGER,
    certifications INTEGER,
    workshops INTEGER,
    talenttests_taken INTEGER,
    olympiads INTEGER,
    reading_writing_skills TEXT,
    memory_capability_score TEXT,
    interested_subjects TEXT,
    interested_career_area TEXT,
    job_or_higher_studies TEXT,
    company_type_preference TEXT,
    taken_inputs_from_seniors TEXT,
    interested_in_games TEXT,
    interested_type_of_books TEXT,
    salary_range_expected TEXT,
    relationship_status TEXT,
    behavior_type TEXT,
    management_or_technical TEXT,
    salary_or_work_preference TEXT,
    work_style TEXT,
    worked_in_teams TEXT,
    introvert TEXT,
    suggested_job_role TEXT
)
'''

In [35]:
conn = create_database_connection()
cursor = conn.cursor()

In [37]:
# Executing the create table commands
cursor.execute(create_jobs_table)
cursor.execute(create_salary_table)
cursor.execute(create_career_table)
conn.commit()

In [39]:
def import_data_to_tables(conn, salary_data, roo_data, roles_skills_data):
    try:
        # Import roles and skills data
        roles_skills_data.to_sql('job_skills_mapping', conn, if_exists='replace', index=False)
        
        # Import salary data
        salary_data.to_sql('salary_prediction', conn, if_exists='replace', index=False)
        
        # Rename columns for career_prediction table to match schema
        career_data = roo_data.copy()
        career_data.columns = [
            'academic_os_percentage',
            'algorithms_percentage',
            'programming_concepts_percentage',
            'software_engineering_percentage',
            'computer_networks_percentage',
            'electronics_subjects_percentage',
            'computer_architecture_percentage',
            'mathematics_percentage',
            'communication_skills_percentage',
            'hours_working_per_day',
            'logical_quotient_rating',
            'hackathons',
            'coding_skills_rating',
            'public_speaking_points',
            'can_work_long_time',
            'self_learning_capability',
            'extra_courses_done',
            'certifications',
            'workshops',
            'talenttests_taken',
            'olympiads',
            'reading_writing_skills',
            'memory_capability_score',
            'interested_subjects',
            'interested_career_area',
            'job_or_higher_studies',
            'company_type_preference',
            'taken_inputs_from_seniors',
            'interested_in_games',
            'interested_type_of_books',
            'salary_range_expected',
            'relationship_status',
            'behavior_type',
            'management_or_technical',
            'salary_or_work_preference',
            'work_style',
            'worked_in_teams',
            'introvert',
            'suggested_job_role'
        ]
        
        # Import to career_prediction table
        career_data.to_sql('career_prediction', conn, if_exists='replace', index=False)
        
        print("Data imported successfully to all tables!")
        return True
        
    except Exception as e:
        print(f"Error importing data to tables: {e}")
        return False

In [41]:
# Import data
success = import_data_to_tables(conn, salary_data, roo_data, roles_skills_data)

Data imported successfully to all tables!


In [43]:
def verify_table_data(conn):
    cursor = conn.cursor()
    
    tables = ['job_skills_mapping', 'salary_prediction', 'career_prediction']
    
    for table in tables:
        # Get record count
        cursor.execute(f"SELECT COUNT(*) FROM {table}")
        count = cursor.fetchone()[0]
        
        # Get sample records
        cursor.execute(f"SELECT * FROM {table} LIMIT 3")
        sample = cursor.fetchall()
        
        print(f"\nTable: {table}")
        print(f"Total records: {count}")
        print("Sample records:")
        for record in sample:
            print(record)

In [45]:
if success:
    # Verify the import
    verify_table_data(conn)


Table: job_skills_mapping
Total records: 74
Sample records:
('AI Ethics Consultant', 'Ethics, AI Governance', 'Mid-Level, Senior')
('AI/ML Engineer', 'TensorFlow, Python, AI/ML', 'Mid-Level')
('Blockchain Developer', 'Solidity, Blockchain', 'Mid-Level')

Table: salary_prediction
Total records: 375
Sample records:
(32.0, 'Male', "Bachelor's", 'Software Engineer', 5.0, 90000.0)
(28.0, 'Female', "Master's", 'Data Analyst', 3.0, 65000.0)
(45.0, 'Male', 'PhD', 'Senior Manager', 15.0, 150000.0)

Table: career_prediction
Total records: 20000
Sample records:
(69, 63, 78, 87, 94, 94, 87, 84, 61, 9, 4, 0, 4, 8, 'yes', 'yes', 'yes', 'shell programming', 'cloud computing', 'no', 'yes', 'excellent', 'excellent', 'cloud computing', 'system developer', 'higherstudies', 'Web Services', 'no', 'no', 'Prayer books', 'salary', 'no', 'stubborn', 'Management', 'salary', 'hard worker', 'yes', 'no', 'Database Developer')
(78, 62, 73, 60, 71, 70, 73, 84, 91, 12, 7, 1, 2, 3, 'yes', 'no', 'yes', 'machine learni

### Data integration Check

### 1. Null values check

In [49]:
def check_basic_data_quality(conn):
    """Check for nulls, duplicates, and invalid values"""
    
    print("BASIC DATA QUALITY CHECKS\n")
    
    # Create connection to reuse queries
    cursor = conn.cursor()
    
    # Check all tables for null values
    tables = ['career_prediction', 'salary_prediction', 'job_skills_mapping']
    
    for table in tables:
        print(f"\nChecking {table}:")
        df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
        
        # Null check
        null_counts = df.isnull().sum()
        if null_counts.any():
            print("Null values found:")
            print(null_counts[null_counts > 0])
        else:
            print("No null values found")
            
        # Duplicate check
        duplicates = df.duplicated().sum()
        print(f"Number of duplicate rows: {duplicates}")

In [51]:
check_basic_data_quality(conn)

BASIC DATA QUALITY CHECKS


Checking career_prediction:
No null values found
Number of duplicate rows: 0

Checking salary_prediction:
Null values found:
age                    2
gender                 2
education_level        2
job_title              2
years_of_experience    2
salary                 2
dtype: int64
Number of duplicate rows: 50

Checking job_skills_mapping:
No null values found
Number of duplicate rows: 0


### 2. Numeric fields validation like percentages and ages etc..

In [53]:
def check_numeric_constraints(conn):
    """Validate numeric fields are within expected ranges"""
    
    print("\nNUMERIC FIELD VALIDATIONS\n")
    
    # Career Prediction numeric validations
    query = """
    SELECT COUNT(*) as invalid_count
    FROM career_prediction
    WHERE 
        academic_os_percentage NOT BETWEEN 0 AND 100
        OR algorithms_percentage NOT BETWEEN 0 AND 100
        OR programming_concepts_percentage NOT BETWEEN 0 AND 100
        OR logical_quotient_rating NOT BETWEEN 0 AND 10
        OR coding_skills_rating NOT BETWEEN 0 AND 10
    """
    
    df = pd.read_sql_query(query, conn)
    print("Invalid percentage/rating values in career_prediction:", df['invalid_count'][0])
    
    # Salary Prediction numeric validations
    query = """
    SELECT COUNT(*) as invalid_count
    FROM salary_prediction
    WHERE 
        age < 18 
        OR age > 100
        OR years_of_experience < 0
        OR years_of_experience > age - 18
        OR salary <= 0
    """
    
    df = pd.read_sql_query(query, conn)
    print("Invalid numeric values in salary_prediction:", df['invalid_count'][0])

In [56]:
check_numeric_constraints(conn)


NUMERIC FIELD VALIDATIONS

Invalid percentage/rating values in career_prediction: 0
Invalid numeric values in salary_prediction: 0


### 3. categorical field validations

In [59]:
def check_categorical_constraints(conn):
    """Validate categorical fields contain expected values"""
    
    print("\nCATEGORICAL FIELD VALIDATIONS\n")
    
    # Check gender values in salary_prediction
    query = """
    SELECT DISTINCT gender 
    FROM salary_prediction
    WHERE gender NOT IN ('Male', 'Female')
    """
    invalid_gender = pd.read_sql_query(query, conn)
    print("Invalid gender values:", invalid_gender.values.tolist())
    
    # Check education_level values
    query = """
    SELECT DISTINCT education_level 
    FROM salary_prediction
    WHERE education_level NOT IN ('Bachelor''s', 'Master''s', 'PhD')
    """
    invalid_edu = pd.read_sql_query(query, conn)
    print("Invalid education levels:", invalid_edu.values.tolist())
    
    # Check yes/no fields in career_prediction
    yes_no_columns = ['can_work_long_time', 'self_learning_capability', 
                     'worked_in_teams', 'introvert']
    
    for column in yes_no_columns:
        query = f"""
        SELECT DISTINCT {column}
        FROM career_prediction
        WHERE {column} NOT IN ('yes', 'no')
        """
        invalid_values = pd.read_sql_query(query, conn)
        print(f"Invalid values in {column}:", invalid_values.values.tolist())

In [61]:
check_categorical_constraints(conn)


CATEGORICAL FIELD VALIDATIONS

Invalid gender values: []
Invalid education levels: []
Invalid values in can_work_long_time: []
Invalid values in self_learning_capability: []
Invalid values in worked_in_teams: []
Invalid values in introvert: []


### 4. Referential integrity

In [64]:
def check_referential_integrity(conn):
    """Check for referential integrity between tables"""
    
    print("\nREFERENTIAL INTEGRITY CHECKS\n")
    
    # Check if job titles in salary_prediction exist in job_skills_mapping
    query = """
    SELECT DISTINCT sp.job_title
    FROM salary_prediction sp
    LEFT JOIN job_skills_mapping jsm ON sp.job_title = jsm.job_title
    WHERE jsm.job_title IS NULL
    """
    
    mismatched_jobs = pd.read_sql_query(query, conn)
    print("Job titles in salary_prediction not found in job_skills_mapping:")
    print(mismatched_jobs.values.tolist())
    
    # Check if suggested job roles in career_prediction exist in job_skills_mapping
    query = """
    SELECT DISTINCT cp.suggested_job_role
    FROM career_prediction cp
    LEFT JOIN job_skills_mapping jsm ON cp.suggested_job_role = jsm.job_title
    WHERE jsm.job_title IS NULL
    """
    
    mismatched_roles = pd.read_sql_query(query, conn)
    print("\nSuggested job roles not found in job_skills_mapping:")
    print(mismatched_roles.values.tolist())

In [66]:
check_referential_integrity(conn)


REFERENTIAL INTEGRITY CHECKS

Job titles in salary_prediction not found in job_skills_mapping:
[['Senior Manager'], ['Sales Associate'], ['Director'], ['Marketing Analyst'], ['Product Manager'], ['Sales Manager'], ['Marketing Coordinator'], ['Senior Scientist'], ['HR Manager'], ['Financial Analyst'], ['Project Manager'], ['Customer Service Rep'], ['Operations Manager'], ['Marketing Manager'], ['Senior Engineer'], ['Data Entry Clerk'], ['Sales Director'], ['VP of Operations'], ['IT Support'], ['Recruiter'], ['Financial Manager'], ['Social Media Specialist'], ['Software Manager'], ['Junior Developer'], ['Senior Consultant'], ['Product Designer'], ['CEO'], ['Accountant'], ['Marketing Specialist'], ['Technical Writer'], ['HR Generalist'], ['Project Engineer'], ['Customer Success Rep'], ['Sales Executive'], ['Operations Director'], ['Administrative Assistant'], ['Strategy Consultant'], ['Copywriter'], ['Account Manager'], ['Director of Marketing'], ['Help Desk Analyst'], ['Customer Service

### Data Cleaning

In [69]:
def clean_salary_prediction_table(conn):
    """Clean salary prediction table by removing nulls and duplicates"""
    
    # Read the current data
    query = "SELECT * FROM salary_prediction"
    df = pd.read_sql_query(query, conn)
    
    print("Original salary prediction records:", len(df))
    
    # Remove null values
    df = df.dropna()
    print("Records after removing nulls:", len(df))
    
    # Remove duplicates
    df = df.drop_duplicates()
    print("Records after removing duplicates:", len(df))
    
    # Update the table
    df.to_sql('salary_prediction', conn, if_exists='replace', index=False)
    
    return df

In [71]:
# Clean salary prediction table
cleaned_salary = clean_salary_prediction_table(conn)

Original salary prediction records: 375
Records after removing nulls: 373
Records after removing duplicates: 324


In [75]:
def verify_cleaning(conn):
    """Verify the cleaning operations"""
    
    print("\nVERIFYING CLEANED DATA:")
    
    # Check for nulls
    query = """
    SELECT 
        (SELECT COUNT(*) FROM salary_prediction WHERE job_title IS NULL) as salary_nulls,
        (SELECT COUNT(*) FROM career_prediction WHERE suggested_job_role IS NULL) as career_nulls
    """
    null_check = pd.read_sql_query(query, conn)
    print("\nNull values remaining:")
    print(null_check)
    
    # Check for duplicates
    query = """
    SELECT 
        COUNT(*) - COUNT(DISTINCT job_title) as duplicate_titles 
    FROM salary_prediction
    """
    duplicate_check = pd.read_sql_query(query, conn)
    print("\nDuplicate job titles:", duplicate_check.iloc[0,0])
    
    # Recheck referential integrity
    query = """
    SELECT COUNT(DISTINCT sp.job_title) as mismatched_titles
    FROM salary_prediction sp
    LEFT JOIN job_skills_mapping jsm ON sp.job_title = jsm.job_title
    WHERE jsm.job_title IS NULL
    """
    integrity_check = pd.read_sql_query(query, conn)
    print("\nMismatched job titles after cleaning:", integrity_check.iloc[0,0])

In [77]:
# Verify the cleaning
verify_cleaning(conn)


VERIFYING CLEANED DATA:

Null values remaining:
   salary_nulls  career_nulls
0             0             0

Duplicate job titles: 150

Mismatched job titles after cleaning: 165
